<a href="https://colab.research.google.com/github/skagnihotri/MALE-FEMALE-CLASSIFICATION/blob/master/male_female_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MALE FEMALE CLASSIFICATION

## Loading Data

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"smagni","key":"b4fd8d61cfa7a806a19e09267e4f31d6"}'}

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d playlist/men-women-classification

 96% 583M/606M [00:02<00:00, 250MB/s]
100% 606M/606M [00:02<00:00, 229MB/s]


In [0]:
!unzip /content/men-women-classification.zip

In [0]:
from shutil import move
!mkdir training
move("men", "training")
move("women", "training")

'training/women'

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image

In [0]:
X = []
y = []
for category in os.listdir("/content/training"):
    print(category)
    for img in os.listdir(f"/content/training/{category}"):
        test_image = image.load_img(f"/content/training/{category}/{img}", target_size= (150, 150))
        test_image = image.img_to_array(test_image)
        X.append(test_image)
        y.append(category)

women
men


In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

X_train = np.array(X)
X_train = X_train / 255.0 
y_train = np.array(y)
label_y = LabelEncoder()
y_train = label_y.fit_transform(y_train)
X_train, y_train = shuffle(X_train, y_train)
X_train.shape, y_train.shape

((3330, 150, 150, 3), (3330,))

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size= 0.2)
X_train.shape, X_test.shape

((2664, 150, 150, 3), (666, 150, 150, 3))

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False,
                             rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip = True)
datagen.fit(X_train)

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow import keras

In [0]:
model = Sequential()

model.add(Conv2D(filters= 64, kernel_size= (3, 3), strides= 1, padding= "same" , activation= 'relu', input_shape= (150, 150, 3)))
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(Conv2D(filters= 64, kernel_size= (3, 3), strides= 1, padding= "same", activation= 'relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Flatten())

model.add(Dense(units= 1024, activation= 'relu'))
# model.add(Dropout(rate= 0.2))
model.add(Dense(units= 512, activation= 'relu'))
# model.add(Dropout(rate= 0.2))
model.add(Dense(units= 1, activation= "sigmoid"))

In [0]:
model.compile(optimizer= 'adam', loss= "binary_crossentropy", metrics=["accuracy"])

In [0]:
model.fit_generator(datagen.flow(X_train, y_train, batch_size=32), steps_per_epoch=len(X_train) / 32, epochs= 50, validation_data=(X_test, y_test))

Epoch 1/50
84/83 [==============================] - 13s 151ms/step - loss: 0.8561 - acc: 0.5544 - val_loss: 0.6783 - val_acc: 0.5781
Epoch 2/50
84/83 [==============================] - 11s 131ms/step - loss: 0.6777 - acc: 0.5709 - val_loss: 0.6668 - val_acc: 0.5796
Epoch 3/50
84/83 [==============================] - 11s 130ms/step - loss: 0.6624 - acc: 0.6066 - val_loss: 0.6481 - val_acc: 0.6517
Epoch 4/50
84/83 [==============================] - 11s 129ms/step - loss: 0.6445 - acc: 0.6445 - val_loss: 0.6177 - val_acc: 0.6922
Epoch 5/50
84/83 [==============================] - 11s 131ms/step - loss: 0.6273 - acc: 0.6456 - val_loss: 0.5917 - val_acc: 0.6832
Epoch 6/50
84/83 [==============================] - 11s 125ms/step - loss: 0.6295 - acc: 0.6584 - val_loss: 0.6019 - val_acc: 0.6892
Epoch 7/50
84/83 [==============================] - 11s 129ms/step - loss: 0.6133 - acc: 0.6637 - val_loss: 0.6075 - val_acc: 0.6712
Epoch 8/50
84/83 [==============================] - 11s 131ms/step - 

In [0]:
model.evaluate(X_test, y_test)

666/666 [==============================] - 0s 615us/sample - loss: 0.5319 - acc: 0.7477


[0.5319278430831325, 0.7477477]